### 导入库

In [1]:
import numpy as np
import pandas as pd

### 修改工作目录

In [2]:
# file_add = r'D:\小鸡理财\百度云同步盘\小鸡理财\数据报告\信息披露'
# file_name = '7月数据披露.xlsx'
# os.chdir(file_add)

### 导入excel

In [3]:
with pd.ExcelFile(r'D:\小鸡理财\百度云同步盘\小鸡理财\数据报告\信息披露\10月数据披露.xlsx') as xlsx:
    df_tz = pd.read_excel(xlsx,'当月投资')
    df_jd_bj = pd.read_excel(xlsx,'经典本金')
    df_jd_lx = pd.read_excel(xlsx,'经典利息')
    df_cg = pd.read_excel(xlsx,'存管回款')
    df_2018_cj = pd.read_excel(xlsx,'2018累计出借人数量')
    df_ljcj = pd.read_excel(xlsx,'累计出借人数量')

- 累计借贷金额(万元)*

In [5]:
#本月借贷金额
cheng_jiao_sum = df_tz['总投资额'].sum()
print('本月借贷金额：',cheng_jiao_sum)

本月借贷金额： 59914600.0


In [6]:
last = input('上月累计借贷金额')
ljjd_sum = float(last) + cheng_jiao_sum

print('\n累计借贷金额：上月{0} + 本月{1} = {2}'\
      .format(float(last),cheng_jiao_sum,ljjd_sum))

print('\n累计借贷金额(万元):{:.0f}'.format(ljjd_sum/10000))

上月累计借贷金额2518168084.03

累计借贷金额：上月2518168084.03 + 本月59914600.0 = 2578082684.03

累计借贷金额(万元):257808


- 累计出借笔数(笔)*

In [8]:
#本月出借笔数
cheng_jiao_count = df_tz['总投资额'].count()
print('本月出借笔数：',cheng_jiao_count)

本月出借笔数： 13692


In [9]:
last_count = input('上月累计出借笔数')

print('\n累计出借笔数：上月{0} + 本月{1} = {2}'\
      .format(int(last_count),cheng_jiao_count,int(last_count) + cheng_jiao_count))

上月累计出借笔数310649

累计出借笔数：上月310649 + 本月13692 = 324341


- 借贷余额（万元）-本金

In [10]:
df_cg_bj = df_cg.loc[:,['身份证','投资时间','预计本次发放时间','本次发放本金']]
# df_cg_bj.head()

df_bj = df_jd_bj.append(df_cg_bj, ignore_index=True,sort=True)
# df_bj.info()

bj_sum = df_bj['本次发放本金'].sum()
print('借贷余额:',bj_sum)
print('借贷余额（万元）:{:.0f}'.format(bj_sum/10000))

借贷余额: 299079350.50000006
借贷余额（万元）:29908


- 借贷余额笔数(笔)

In [22]:
df_bj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200314 entries, 0 to 200313
Data columns (total 4 columns):
投资时间        200314 non-null object
本次发放本金      200314 non-null float64
身份证         200314 non-null object
预计本次发放时间    200314 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 6.1+ MB


In [24]:
df_tz.shape

(13692, 51)

In [23]:
df_tz.drop_duplicates(subset=['身份证','投资时间']).shape

(13070, 51)

In [11]:
df_yebs = df_bj.drop_duplicates(subset = ['投资时间','身份证'])
print('借贷余额笔数:',df_yebs.shape[0])

借贷余额笔数: 36790


- 利息余额(万元)

In [12]:
df_cg_lx = df_cg.loc[:,['总利息','投资时间','预计本次发放时间','身份证']]
# df_cg_bj.head()

df_lx = df_jd_lx.append(df_cg_lx, ignore_index=True,sort=True)
# df_lx.info()

bj_lx = df_lx['总利息'].sum()
print('利息余额:',bj_lx)
print('利息余额（万元）:{:.0f}'.format(bj_lx/10000))

利息余额: 16853557.810000002
利息余额（万元）:1685


- 累计出借人数量(人)

In [13]:
df_ljcj_new = df_ljcj.append(df_cg['身份证'].to_frame())
# df_ljcj_new.info()

#检查身份证是否有小写x，会影响到去重计算
while df_ljcj_new[df_ljcj_new['身份证'].str.contains('x')].shape[0]:
    df_ljcj_new['身份证'] = df_ljcj_new['身份证'].str.upper()

df_ljcj_new.drop_duplicates(inplace=True)
print('累计出借人数量:',df_ljcj_new.shape[0])

df_ljcj_new.to_excel('累计出借人数量.xlsx', sheet_name='Sheet1',index=False)

累计出借人数量: 37470


- 当前出借人数量(人)

In [14]:
df_dqcj_new = df_jd_bj['身份证'].append([df_jd_lx['身份证'],df_cg['身份证']])

#检查身份证是否有小写x，会影响到去重计算
while df_dqcj_new[df_dqcj_new.str.contains('x')].shape[0]:
    df_dqcj_new = df_dqcj_new.str.upper()


df_dqcj_new.drop_duplicates(inplace=True)
print('当前出借人数量:',df_dqcj_new.shape[0])

当前出借人数量: 4360


- 人均累计出借金额（万元）

In [15]:
print('人均累计出借金额:{}'.format(ljjd_sum / df_ljcj_new.shape[0]))
print('\n人均累计出借金额（万元）:{:.2f}'.format(ljjd_sum / df_ljcj_new.shape[0] / 10000))

人均累计出借金额:68803.91470589806

人均累计出借金额（万元）:6.88


- 最大单户出借余额占比

In [16]:
#检查身份证是否有小写x，会影响到计算
while df_bj[df_bj['身份证'].str.contains('x')].shape[0]:
    df_bj['身份证'] = df_bj['身份证'].str.upper()

gp_bj = df_bj.groupby('身份证',as_index=False)['本次发放本金'].sum().sort_values(by='本次发放本金',ascending=False)
# gp_bj

print('最大单户出借余额占比:{:.2%}'.format(gp_bj.iloc[0,1] / df_bj['本次发放本金'].sum()))

最大单户出借余额占比:5.02%


- 最大十户出借余额占比

In [17]:
gp_bj['ranking'] = gp_bj['本次发放本金'].rank(method='min',ascending=False)
# gp_bj

first_10_sum = gp_bj[gp_bj['ranking'] < 11]['本次发放本金'].sum()
print('最大十户出借余额占比:{:.2%}'.format(first_10_sum / df_bj['本次发放本金'].sum()))

最大十户出借余额占比:11.51%


- 当月出借人数量

In [19]:
cj_pp_sum = df_tz['身份证'].nunique()

print('当月出借人数量 : {0} 人'.format(cj_pp_sum))

当月出借人数量 : 1889 人


- 2018年出借人数量

In [21]:
df_2018_cj_new = df_2018_cj['身份证号码'].append(df_tz['身份证'])

df_2018_cj_new = df_2018_cj_new.str.upper().drop_duplicates()

df_2018_cj_sum = df_2018_cj_new.shape[0]

print('2018年出借人数量 : {0} 人'.format(df_2018_cj_sum))

df_2018_cj_new.to_excel('2018年出借人.xlsx')

2018年出借人数量 : 7013 人
